# CRIMS


In [ ]:
# Llibreries
# Lybraries

import os
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
import geopandas as gpd

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt


In [ ]:
# Carregar les dades
# Load data

df24 = pd.read_csv('CMX_2024.csv', encoding='latin1')

print(df24.columns)

# Eliminar variables innecesaries
# Drop unnecessary variables

df24 = df24.drop(['anio_inicio','mes_inicio','fecha_inicio','hora_inicio', 'anio_hecho'], axis=1)

df24 = df24.drop(['competencia', 'fiscalia', 'agencia',
       'unidad_investigacion', 'colonia_hecho', 'colonia_catalogo',
       'alcaldia_hecho', 'alcaldia_catalogo', 'municipio_hecho', ], axis=1)

df24 = df24[df24['categoria_delito'] != 'HECHO NO DELICTIVO']

display(df24.info())

In [ ]:
# encoding

df24_BE = df24.copy()

print(df24.info())

# Encoding categorical variables
label_encoder = LabelEncoder()
df24['mes_hecho'] = label_encoder.fit_transform(df24['mes_hecho'])
df24['delito_encoded'] = label_encoder.fit_transform(df24['delito'])
df24['categoria_delito_encoded'] = label_encoder.fit_transform(df24['categoria_delito'])
df24['fecha_hecho'] = label_encoder.fit_transform(df24['fecha_hecho'])


# Convertir a formato timedelta
df24['hora_hecho'] = pd.to_timedelta(df24['hora_hecho'])
# Convertir a segundos
df24['time_seconds'] = df24['hora_hecho'].dt.total_seconds()
# Drop the 'hora_hecho' column
df24 = df24.drop('hora_hecho', axis=1)


print(df24.info())
print(df24_BE.info())

# Delete missings
df24 = df24.dropna()

print(df24.isnull().sum())


In [ ]:
from sklearn.cluster import KMeans

# Create an instance of the KMeans class
kmeans = KMeans(n_clusters=6)

# Fit the KMeans model to the 'time_seconds' variable
kmeans.fit(df24['time_seconds'].values.reshape(-1, 1))

# Get the cluster labels
cluster_labels = kmeans.labels_

# Assign the cluster labels to the dataframe
df24['time_zones'] = cluster_labels


time_zone_mapping = {0: 'mañana', 1: 'noche', 2: 'madrugada', 3: 'mediodia', 4: 'tarde', 5: 'medianoche'}
df24['time_zones'] = df24['time_zones'].map(time_zone_mapping)
display(df24['time_zones'].value_counts())
# Save the dataframe with time_zones, longitud, and latitud to a CSV file
df24[['time_zones', 'longitud', 'latitud']].to_csv('time_zones_coordinates.csv', index=False)

In [ ]:
df24_1 = df24[df24['time_zones'] == 0]
df24_2 = df24[df24['time_zones'] == 1]
df24_3 = df24[df24['time_zones'] == 2]
df24_4 = df24[df24['time_zones'] == 3]
df24_5 = df24[df24['time_zones'] == 4]
df24_6 = df24[df24['time_zones'] == 5]

In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN

import matplotlib.pyplot as plt

def analyze_clusters(df):
    # Apply the DBSCAN algorithm
    dbscan = DBSCAN(eps=0.5, min_samples=5)
    df2 = df.drop(['delito', 'categoria_delito'], axis=1)
    print(df2.columns)
    df['cluster_label'] = dbscan.fit_predict(df2)

    # Plot the clusters
    plt.figure(figsize=(10, 6))
    plt.scatter(df['latitud'], df['longitud'], c=df['cluster_label'], cmap='viridis')
    plt.xlabel('Latitude')
    plt.ylabel('Longitude')
    plt.title('DBSCAN Clustering')
    plt.show()

    # Count the occurrences of each cluster label
    cluster_counts = df['cluster_label'].value_counts()
    print(cluster_counts)

# analyze_clusters(df24)


In [ ]:
analyze_clusters(df24_1)

cluster_counts = df24_1[['delito', 'cluster_label']]

luster_counts = df24_1.drop(cluster_counts[cluster_counts['cluster_label'] == -1].index)

luster_counts.info()
print(luster_counts.head())

plt.figure(figsize=(10, 6))
luster_counts['delito'].value_counts().plot(kind='bar')
plt.xlabel('Delito')
plt.ylabel('Count')
plt.title('Categorical Variable Plot')
plt.show()

df24_1['time_seconds'].describe()

df24_1.to_csv('CMX_mañana.csv', index=False)


In [ ]:
analyze_clusters(df24_2)

cluster_counts = df24_2[['delito', 'cluster_label']]

luster_counts = df24_2.drop(cluster_counts[cluster_counts['cluster_label'] == -1].index)

luster_counts.info()
print(luster_counts.head())

plt.figure(figsize=(10, 6))
luster_counts['delito'].value_counts().plot(kind='bar')
plt.xlabel('Delito')
plt.ylabel('Count')
plt.title('Categorical Variable Plot')
plt.show()

df24_2['time_seconds'].describe()

df24_2.to_csv('CMX_noche.csv', index=False)

In [ ]:
analyze_clusters(df24_3)

cluster_counts = df24_3[['delito', 'cluster_label']]

luster_counts = df24_3.drop(cluster_counts[cluster_counts['cluster_label'] == -1].index)

luster_counts.info()
print(luster_counts.head())

plt.figure(figsize=(10, 6))
luster_counts['delito'].value_counts().plot(kind='bar')
plt.xlabel('Delito')
plt.ylabel('Count')
plt.title('Categorical Variable Plot')
plt.show()

df24_3['time_seconds'].describe()

df24_3.to_csv('CMX_madrugada.csv', index=False)

In [ ]:
analyze_clusters(df24_4)

cluster_counts = df24_4[['delito', 'cluster_label']]

luster_counts = df24_4.drop(cluster_counts[cluster_counts['cluster_label'] == -1].index)

luster_counts.info()
print(luster_counts.head())

plt.figure(figsize=(10, 6))
luster_counts['delito'].value_counts().plot(kind='bar')
plt.xlabel('Delito')
plt.ylabel('Count')
plt.title('Categorical Variable Plot')
plt.show()

df24_4['time_seconds'].describe()

df24_4.to_csv('CMX_mediodia.csv', index=False)

In [ ]:
analyze_clusters(df24_5)

cluster_counts = df24_5[['delito', 'cluster_label']]

luster_counts = df24_5.drop(cluster_counts[cluster_counts['cluster_label'] == -1].index)

luster_counts.info()
print(luster_counts.head())

plt.figure(figsize=(10, 6))
luster_counts['delito'].value_counts().plot(kind='bar')
plt.xlabel('Delito')
plt.ylabel('Count')
plt.title('Categorical Variable Plot')
plt.show()

df24_5['time_seconds'].describe()

df24_5.to_csv('CMX_tarde.csv', index=False)

In [ ]:
analyze_clusters(df24_6)

cluster_counts = df24_6[['delito', 'cluster_label']]

luster_counts = df24_6.drop(cluster_counts[cluster_counts['cluster_label'] == -1].index)

luster_counts.info()
print(luster_counts.head())

plt.figure(figsize=(10, 6))
luster_counts['delito'].value_counts().plot(kind='bar')
plt.xlabel('Delito')
plt.ylabel('Count')
plt.title('Categorical Variable Plot')
plt.show()

df24_6['time_seconds'].describe()

df24_6.to_csv('CMX_medianoche.csv', index=False)

In [ ]:
import geopandas as gpd

# Create a GeoDataFrame from the DataFrame with cluster labels
gdf_cluster = gpd.GeoDataFrame(df24, geometry=gpd.points_from_xy(df24['longitud'], df24['latitud']))

# Save the GeoDataFrame as a GeoJSON file
gdf_cluster.to_file('cluster_data.geojson', driver='GeoJSON')

# Plot the GeoDataFrame on a map with different colors for each cluster
gdf_cluster.plot(column='cluster_label', legend=True)

In [ ]:
print(df24_1['time_seconds'].max())
print(df24_2['time_seconds'].max())
print(df24_3['time_seconds'].max())
print(df24_4['time_seconds'].max())
print(df24_5['time_seconds'].max())
print(df24_6['time_seconds'].max())